# Perform EDA on raw data files

In [10]:
%cd /Volumes/External/source/GTPracticum/data 
%ls -salh

/Volumes/External/source/GTPracticum/data
total 7000392
      0 drwxr-xr-x   11 kariato  staff   352B Feb 10 14:58 ./
      0 drwxr-xr-x    7 kariato  staff   224B Feb 10 14:27 ../
     16 -rw-r--r--@   1 kariato  staff   6.0K Feb 10 14:58 .DS_Store
 753984 -rw-r--r--@   1 kariato  staff   358M Feb 10 14:56 archive (1).zip
2814656 -rw-rw-r--@   1 kariato  staff   1.3G Sep 19  2019 emails.csv
2109312 -rw-r--r--    1 kariato  staff   1.0G Feb 10 14:24 enron.sql
 427936 -rw-r--r--@   1 kariato  staff   209M Feb 10 14:24 enron.sql.gz
 885648 -rw-r--r--@   1 kariato  staff   423M Feb 10 14:35 enron_mail_20150507.tar.gz
      0 drwxr-xr-x@  11 kariato  staff   352B Nov 10  2004 enron_with_categories/
   8840 -rw-r--r--@   1 kariato  staff   4.3M Feb 10 14:25 enron_with_categories.tar.gz
      0 drwxr-xr-x@ 152 kariato  staff   4.8K Apr  1  2011 maildir/


In [ ]:
!tar -xvf enron_mail_20150507.tar.gz

In [6]:
!gzip -dk enron.sql.gz

In [ ]:
!tar -xvf enron_with_categories.tar.gz

In [12]:
import pandas as pd
enron=pd.read_csv("emails.csv")

In [28]:

    print(email_row['file'])
    print(email_row['message'])
    break

In [67]:
email_tags=['Subject','To','CC','BCC','SENT','From','Date','Content-Type','Message-ID','Mime-Version','Content-Transfer-Encoding',
            'X-From','X-To','X-cc','X-bcc','X-Folder','X-Origin','X-FileName']
email_tags=[i.upper() for i in email_tags]

In [68]:
from collections import defaultdict
tags=defaultdict(int)
email_list=[]
for email_header,email_row in enron.iterrows():
    email_txt=email_row["message"]
    body=[]
    email_header={}
    for email_line in email_txt.splitlines():
        header_parts=email_line.split(":")
        if len(header_parts)>1:
            tag=header_parts[0].upper().strip()
            if  tag in email_tags:
                if tag in email_header:
                    email_header[tag]+='>> '+':'.join(header_parts[1:-1])
                else: 
                    email_header[tag]=':'.join(header_parts[1:-1])
                if header_parts[0]=='Subject':
                    body.append(email_line)
            else:
                body.append(email_line)                
                tags[tag]+=1
        else:
            body.append(email_line)
    email_header['body']='\n'.join(body)
    email_header['raw']=email_txt
    email_list.append(email_header)
tagsEdited={i:j for i,j in tags.items() if j>400}
email_df=pd.DataFrame(email_list)
bad_tag=pd.DataFrame([tagsEdited]).T.sort_values(0,ascending=False)

In [69]:
bad_tag.reset_index()  

,index,0
0,HTTP,138959
1,<HTTP,23086
2,FAX,20727
3,TIME,17056
4,ALIAS,13667
...,...,...
325,STAR WARS,405
326,=09SENT BY,405
327,REPLY-TO,404
328,CONFIRMATION,402


In [58]:
email_df.head(1).T

,0
body,Message-ID: <18782981.1075855378110.JavaMail.e...
raw,Message-ID: <18782981.1075855378110.JavaMail.e...


In [74]:
email_df2=email_df.sample(1000)
print(email_df2['body'].tail(1).to_list()[0])

Subject: ETS Products/Services

Please follow up with Lou on this.  It sounds like something the commercial 
people would be in a good position to answer, so maybe start by finding out 
if they are already responding, either through Lou or otherwise.  Then get 
the right people to review and supplement this list.  Thanks. df 
---------------------- Forwarded by Drew Fossum/ET&S/Enron on 11/08/2000 
12:29 PM ---------------------------

11/08/2000 11:06 AM
Louis Soldano
Louis Soldano
Louis Soldano
11/08/2000 11:06 AM
11/08/2000 11:06 AM
Sample/Houston/Eott@Eott, David Johnson/EU/Enron@Enron, David L 
Johnson/OTS/Enron@ENRON, David Bush/ECF/Enron@ENRON, Mark 
Cartwright/ECF/Enron@ENRON, Drew Fossum/ET&S/Enron@ENRON, Dorothy 
McCoppin/FGT/Enron@ENRON, Don Hawkins/OTS/Enron@Enron
Prentice/GPGFIN/Enron@ENRON, Colleen Raker/ET&S/Enron@ENRON, Michael 
Moran/ET&S/Enron@ENRON 

Subject: ETS Products/Services

EHS Corporate has asked we verify this inventory of services/products  from 
all of th

In [77]:
print(email_df2['raw'].tail(1).to_list()[0])

Message-ID: <12110590.1075842506433.JavaMail.evans@thyme>
Date: Wed, 8 Nov 2000 04:31:00 -0800 (PST)
From: drew.fossum@enron.com
To: maria.pavlou@enron.com
Subject: ETS Products/Services
Cc: louis.soldano@enron.com
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
Bcc: louis.soldano@enron.com
X-From: Drew Fossum
X-To: Maria Pavlou
X-cc: Louis Soldano
X-bcc: 
X-Folder: \Drew_Fossum_Dec2000_June2001_1\Notes Folders\Sent
X-Origin: FOSSUM-D
X-FileName: dfossum.nsf

Please follow up with Lou on this.  It sounds like something the commercial 
people would be in a good position to answer, so maybe start by finding out 
if they are already responding, either through Lou or otherwise.  Then get 
the right people to review and supplement this list.  Thanks. df 
---------------------- Forwarded by Drew Fossum/ET&S/Enron on 11/08/2000 
12:29 PM ---------------------------

11/08/2000 11:06 AM
Louis Soldano
Louis Soldano
Louis Soldano
11/08/2000 11:06 AM
1

In [82]:
print(email_df2['raw'].shape)

(1000,)
